# Calculations implantation depths

In [1]:
%run GenerateStoppingPower.ipynb

/home/yulia/anaconda3/lib/python3.7/site-packages/srim/core/elementdb.py:10: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  return yaml.load(open(dbpath, "r"))


SRIM stopping power successfully calculated
	Calculate energy loss as a function of traversed target material with function 'GetElossAfterX(E_init, x)'
	Calculate traversed target material as a function of energy loss 'GetXAfterEloss(E_init, E_after)'
	Calculate range in material as a function of energy 'GetRange(E_init)'
	Units: MeV, µm
Successfully imported 'GenerateStoppingPower'
	Class to Generate SRIM Stopping Power and calculate energy loss and traversed material.


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import time
import scipy.linalg
from matplotlib import colors
import matplotlib as mpl
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.backends.backend_pdf import PdfPages
from mpl_toolkits.axes_grid1 import make_axes_locatable
import pandas as pd

color_list = plt.rcParams['axes.prop_cycle'].by_key()['color'];
#Customising style
#mpl.rc('lines', linewidth=2, color='r')
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['font.size'] = 24
#mpl.rcParams['savefig.format'] = "eps"
#mpl.rcParams['savefig.dpi'] = 100
#mpl.rcParams['text.usetex'] = True
#mpl.rcParams['figure.figsize'] = (16,9)

## Material

In [3]:
# Construct a layer of SiO2, arbitrary width (as it is not relevant for these calcs)
layer = Layer({
    'Si': {
        'stoich': 1,
        #'E_d': 35.0, # Displacement Energy
        #'lattice': 0.0,
        #'surface': 3.0
    },
    'O': {
        'stoich': 2,
        #'E_d': 20.0, # Displacement Energy
        #'lattice': 0.0,
        #'surface': 3.0
    }
}, density=2.32, width=10000.0)

## Fast-Alphas

The kinematics as calculated in [notebook](TASCA_U225.ipynb) is assumed.

Ion: U-225

In [4]:
E_kin = 45 #MeV

In [5]:
# Construct ion
A, E_per_A_max = 225, 7e6 # nucleon number, eV 
energy_max = A*E_per_A_max
#print("Max energy=", energy_max)
ion = Ion('U', energy=energy_max, mass=A)

U225 = GenerateStoppingPower(layer, ion)

SRIM stopping power successfully calculated
	Calculate energy loss as a function of traversed target material with function 'GetElossAfterX(E_init, x)'
	Calculate traversed target material as a function of energy loss 'GetXAfterEloss(E_init, E_after)'
	Calculate range in material as a function of energy 'GetRange(E_init)'
	Units: MeV, µm


In [6]:
print("Range =", U225.GetRange(45, from_sr=True), "µm")

Range = 7.429001481209475 µm


## Fl-289

In [7]:
# Construct ion
A, E_per_A_max = 289, 7e6 # nucleon number, eV 
energy_max = A*E_per_A_max
#print("Max energy=", energy_max)
ion = Ion('U', energy=energy_max, mass=A)

Fl289 = GenerateStoppingPower(layer, ion, z_ion=114)

Extrapolating stopping power for ion with z= 114
Mean scaling factor: 1.3309325475011262
SRIM stopping power successfully calculated
	Calculate energy loss as a function of traversed target material with function 'GetElossAfterX(E_init, x)'
	Calculate traversed target material as a function of energy loss 'GetXAfterEloss(E_init, E_after)'
	Calculate range in material as a function of energy 'GetRange(E_init)'
	Units: MeV, µm


In [8]:
print("Range =", Fl289.GetRange(22, from_sr=True), "µm")

Range = 3.096210212861479 µm


## Ca-48

Beam-like ions implanting themselves.

In [9]:
# Construct ion
A, E_per_A_max = 48, 7e6 # nucleon number, eV 
energy_max = A*E_per_A_max
#print("Max energy=", energy_max)
ion = Ion('Ca', energy=energy_max, mass=A)

Ca48 = GenerateStoppingPower(layer, ion)

SRIM stopping power successfully calculated
	Calculate energy loss as a function of traversed target material with function 'GetElossAfterX(E_init, x)'
	Calculate traversed target material as a function of energy loss 'GetXAfterEloss(E_init, E_after)'
	Calculate range in material as a function of energy 'GetRange(E_init)'
	Units: MeV, µm


In [10]:
print("Range =", Ca48.GetRange(289, from_sr=True), "µm")

Range = 64.5240132299233 µm


## Alphas

Alphas punching through implantation detector.

In [11]:
# Construct ion
A, E_per_A_max = 4, 100e6 # nucleon number, eV 
energy_max = A*E_per_A_max
#print("Max energy=", energy_max)
ion = Ion('He', energy=energy_max, mass=A)

alpha = GenerateStoppingPower(layer, ion)

SRIM stopping power successfully calculated
	Calculate energy loss as a function of traversed target material with function 'GetElossAfterX(E_init, x)'
	Calculate traversed target material as a function of energy loss 'GetXAfterEloss(E_init, E_after)'
	Calculate range in material as a function of energy 'GetRange(E_init)'
	Units: MeV, µm


In [12]:
print("Range =", alpha.GetRange(30, from_sr=True), "µm")

Range = 398.72438200096076 µm


### Punching through VETO and IMP

Two energy components:

* Beam-like particle collision with He-gas
* Target-like particle collision with He-gas

Assuming elastic collisions.

In [13]:
e_tank1 = 4.774 #MeV/u
e_tank2 = 5.893 #MeV/u
A_beam = 48
A_target = 207

e_tank = e_tank1

Equations for elastic collisions given here: https://en.wikipedia.org/wiki/Elastic_collision

In [14]:
def GetEAfterColl(m1, m2, T):
    T1 = np.multiply(np.square(np.divide(m1-m2,m1+m2)), T)
    T2 = 4*np.multiply(np.divide(np.multiply(m1, m2), np.square(m1+m2)), T)
    
    #T1 = np.square((m1-m2)/(m1+m2))*T
    #T2 = ((4*m1*m2)/np.square(m1+m2))*T
    
    return T1, T2

In [15]:
E_beam = (A_beam*e_tank)

E_target_like, E_tmp = GetEAfterColl(A_beam, A_target, E_beam)

print("E_kin beam-like:", E_beam, "MeV")
print("E_kin beam-like after collision:", E_tmp, "MeV")
print("E_kin target-like after collision:", E_target_like, "MeV")

E_kin beam-like: 229.152 MeV
E_kin beam-like after collision: 140.06023972318337 MeV
E_kin target-like after collision: 89.0917602768166 MeV


In [16]:
E_alpha_from_beam, E_tmp = GetEAfterColl(A_beam, 4, E_beam)
E_alpha_from_target, E_tmp = GetEAfterColl(A_target, 4, E_target_like)

print("E_kin alpha after collision with beam-like:", E_alpha_from_beam, "MeV")
print("E_kin alpha after collision with target-like:", E_alpha_from_target, "MeV")

E_kin alpha after collision with beam-like: 164.06740828402366 MeV
E_kin alpha after collision with target-like: 82.46405851726905 MeV


In [17]:
E_kin_0 = np.asarray([E_alpha_from_beam, E_alpha_from_target]) 

In [18]:
imp_thick = 300 #µm
veto_thick = 300 #µm
dl_imp_veto = 2 #µm

In [19]:
# implantation detector, dead layer
for i, E_kin in enumerate(E_kin_0):
    E_dep = [] #deposited energy for each layer
    E_dep.append(alpha.GetElossAfterX(E_kin, dl_imp_veto)) #deposited energy
    #print(E_dep[-1])
    E_kin -= E_dep[-1] # energy after deposit
    # implantation detector
    E_dep.append(alpha.GetElossAfterX(E_kin, imp_thick-dl_imp_veto)) #deposited energy
    E_kin -= E_dep[-1] # alpha energy after deposit

    # veto detector, dead layer
    E_dep.append(alpha.GetElossAfterX(E_kin, dl_imp_veto)) #deposited energy
    #print(E_dep[-1])
    E_kin -= E_dep[-1] # energy after deposit
    # veto detector
    E_dep.append(alpha.GetElossAfterX(E_kin, veto_thick-dl_imp_veto)) #deposited energy
    E_kin -= E_dep[-1] # alpha energy after deposit
    print(i, "Initial energy:", E_kin_0[i], "MeV\n\tDeposited energies:\n\t", E_dep, "MeV")
    print("\t Energy after:", E_kin, "MeV")

0 Initial energy: 164.06740828402366 MeV
	Deposited energies:
	 [0.022508161890357314, 3.3815756124650136, 0.022885660990823453, 3.4394253335646283] MeV
	 Energy after: 157.20101351511283 MeV
1 Initial energy: 82.46405851726905 MeV
	Deposited energies:
	 [0.038970872946251234, 5.982670129117906, 0.04141276938030103, 6.38575455481913] MeV
	 Energy after: 70.01525019100546 MeV


## IMP and VETOas dE-dE detector

Simulate dE-dE response for different particles.

In [20]:
elms = np.asarray(["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne"])
As = np.asarray(["1", "4", "7", "9", "11", "12", "14", "16", "19", "20"])
#elms = np.asarray(["He"])
#As = np.asarray(["4"])

This may take a while ...

In [21]:
# Construct ions
energy_max = 300e6 #300 MeV

GSP_dEdE = []

for i, elm in enumerate(elms):
    ion = Ion(elm, energy=energy_max, mass=As[i])
    GSP_dEdE.append(GenerateStoppingPower(layer, ion))

SRIM stopping power successfully calculated
	Calculate energy loss as a function of traversed target material with function 'GetElossAfterX(E_init, x)'
	Calculate traversed target material as a function of energy loss 'GetXAfterEloss(E_init, E_after)'
	Calculate range in material as a function of energy 'GetRange(E_init)'
	Units: MeV, µm
SRIM stopping power successfully calculated
	Calculate energy loss as a function of traversed target material with function 'GetElossAfterX(E_init, x)'
	Calculate traversed target material as a function of energy loss 'GetXAfterEloss(E_init, E_after)'
	Calculate range in material as a function of energy 'GetRange(E_init)'
	Units: MeV, µm
SRIM stopping power successfully calculated
	Calculate energy loss as a function of traversed target material with function 'GetElossAfterX(E_init, x)'
	Calculate traversed target material as a function of energy loss 'GetXAfterEloss(E_init, E_after)'
	Calculate range in material as a function of energy 'GetRange(E_ini

In [22]:
imp_thick = 300 #µm
veto_thick = 300 #µm
dl_imp_veto = 5 #µm, dead layer at back and front

Energies in the range [5, 295] MeV is investigated.

In [23]:
energy = np.linspace(20, 295, 1000)

In [24]:
energies_all, E_dep_imp_all, E_dep_veto_all = [], [], [] #deposited energy in imp and box

for i, elm in enumerate(elms):
    print(i, "\tCalculating for particle:", elm)
    #E_dep_imp, E_dep_veto = np.asarray([]), np.asarray([]) #deposited energy for each detector
    E_now = energy
    energies = energy
    # implantation detector, dead layer
    E_dep = GSP_dEdE[i].GetElossAfterX(energy, dl_imp_veto) #deposited energy
    #print(E_dep[-1])
    E_now -= E_dep # energy after deposit
    inds = E_now > 10
    energies = energies[inds]
    E_now = E_now[inds]
    
    # implantation detector
    E_dep = GSP_dEdE[i].GetElossAfterX(E_now, imp_thick-2*dl_imp_veto) #deposited energy
    E_dep_imp_all.append(E_dep) 
    E_now -= E_dep # energy after deposit

    inds = E_now > 10
    energies = energies[inds]
    E_now = E_now[inds]
    E_dep_imp_all[-1] = E_dep_imp_all[-1][inds] 
    
    # imp detector back dead layer and veto detector dead layer
    E_dep = GSP_dEdE[i].GetElossAfterX(E_now, 2*dl_imp_veto) #deposited energy
    #print(E_dep[-1])
    E_now -= E_dep # energy after deposit  
    
    inds = E_now > 10
    energies = energies[inds]
    E_now = E_now[inds]
    E_dep_imp_all[-1] = E_dep_imp_all[-1][inds] 

    # veto detector
    E_dep = GSP_dEdE[i].GetElossAfterX(E_now, imp_thick-2*dl_imp_veto) #deposited energy
    E_dep_veto_all.append(E_dep) 
    E_now -= E_dep # energy after deposit
    energies_all.append(energies)

0 	Calculating for particle: H
1 	Calculating for particle: He
2 	Calculating for particle: Li
3 	Calculating for particle: Be
4 	Calculating for particle: B
5 	Calculating for particle: C
6 	Calculating for particle: N
7 	Calculating for particle: O
8 	Calculating for particle: F
9 	Calculating for particle: Ne


In [25]:
for i, x in enumerate(elms):
    print(energies_all[i].shape, E_dep_imp_all[i].shape, E_dep_veto_all[i].shape)

(1000,) (1000,) (1000,)
(968,) (968,) (968,)
(875,) (875,) (875,)
(768,) (768,) (768,)
(652,) (652,) (652,)
(540,) (540,) (540,)
(395,) (395,) (395,)
(248,) (248,) (248,)
(62,) (62,) (62,)
(0,) (0,) (0,)


In [26]:
%matplotlib widget
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams['font.size'] = 24
plt.figure(figsize=(15, 10))
for i, elm in enumerate(elms):
    plt.plot(E_dep_imp_all[i], E_dep_veto_all[i], label=elm)
plt.xlabel("Energy imp (MeV)")
plt.ylabel("Energy veto (MeV)")
plt.legend()
plt.show()

ModuleNotFoundError: No module named 'ipympl'